In [7]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt

In [8]:

class normalizedDLT:
    
    def __init__(self):
        pass
    
    def similarityTransform(self, X):
        
        x = X[:,0]
        y = X[:,1]
        x_mean = np.mean(x)
        y_mean = np.mean(y)
        num = np.sqrt(2)
        den = 0
        count = len(X)
        
        for i,j in zip(x,y):
            den = den + np.sqrt((i-x_mean)**2) + np.sqrt((j-y_mean)**2)
        
        den = den / count
        s = num / den
        tx = -s * x_mean
        ty = -s * y_mean
        
        T = np.array([[s, 0, tx ], 
                     [0, s, ty ], 
                     [0, 0, 1]])
        return T
    
    def normalize(self, X, T):
        
        normalized_image = []
        for [i,j] in X:
            normalized_image.append(T.dot(np.array([i,j,1])))
        
        normalized_image = np.array(normalized_image)
        return normalized_image
            
    def computeA(self, X1, X2):
        
        A = []
        for i , j in zip(X1, X2):
            x1, y1 = i[0], i[1]
            x2, y2 = j[0], j[1]
            A.append([0 ,0 ,0 , -x1, -y1, -1, y2 * x1, y2 * y1, y2])
            A.append([x1 ,y1 ,1 , 0, 0, 0, -x2 * x1, -x2 * y1, -x2])
        
        A = np.array(A)
        return A

    
    def findHomography(self, X1, X2):
        
        assert(X1.shape == X2.shape)
            
        T1 = self.similarityTransform(X1)
        T2 = self.similarityTransform(X2)
        
        X1 = self.normalize(X1, T1)
        X2 = self.normalize(X2, T2)
        A = self.computeA(X1, X2)
        
        u, s, vh = np.linalg.svd(A)
        v_12 = vh[-1]
        H = v_12.reshape((3,3))
        T2_inv = np.linalg.inv(T2)
        H = T2_inv.dot(H.dot(T1))
        h_33 = H[2,2]
        H = H / h_33
        return H
    

In [9]:
# Read and Process the .txt file
images = dict() 
with open(os.getcwd()+'/boat/homography.txt') as filehandle:
    filecontents = filehandle.readlines()
    i = 0
    for line in filecontents:
        current = (line.split(" = ")[1]).replace("; ",",").replace(" ","")
        j = 1
        image = list()
        while j < len(current) and j+3 < len(current):
            image.append(int(current[j:j+3]))
            j = j + 4
        images[i] = np.array(image).reshape((25,2))   
        i = i + 1  

img1, img2 = images[0], images[1]

In [10]:
image1 = cv2.imread("boat/img1+points.png", cv2.IMREAD_UNCHANGED)
image2 = cv2.imread("boat/img2+points.png", cv2.IMREAD_UNCHANGED)

ndlt = normalizedDLT()
H = ndlt.findHomography(img1,img2)
h = cv2.findHomography(img1, img2)[0]

im_out1 = cv2.warpPerspective(image1, H, (image2.shape[1],image2.shape[0]))
im_out2 = cv2.warpPerspective(image1, h, (image2.shape[1],image2.shape[0]))

cv2.imwrite("handcraftedDLT.jpg",im_out1)
cv2.imwrite("OpenCV.jpg",im_out1)

True